# Tools - Atomic Agentic's Foundational Functional Pattern

Atomic Agentic offers a means to build composable, complex agentic systems using classes specifically built on Gang-of-Four design pattern principles. The center of which revolves a term you may be already somewhat familiar with: **Tools**.

The mainstream concept of a tool overlaps with and is often interchangeable with *methods* and *functions*, but has garnered more association with the two year surge of agentic AI (late 2023 to 2025). Though similar, I propose the following definition for tools in the context of agentic AI and the Atomic-Agentic framework:

> A **Tool** is an *adapter-type interface* for LLM's and humans that provides:
> 1. A pointer to a callable object or method
> 2. A natural language representation of the method's name, parameter-informed signature, and usage/purpose description.

Atomic Agentic provides the `Tool` class, which is designed to offer the added benefit of **a single, schemda-driven, dictionary-based** contract by which a tool can pass inputs to and return the output of its method pointer.

> **All tools are invoked using a single dictionary of inputs.**

## What you’ll learn in this notebook

By the end of this notebook, you should be comfortable with:

- Wrapping a normal Python function as a `Tool`
- Calling that Tool using `.invoke({...})`
- Understanding *why* dict-first invocation is valuable
- Inspecting a Tool’s internal “call plan”
- Understanding where Tool metadata comes from

This notebook will **only focus on tools created from local python methods**, but we will cover how we've adapted the standard tool template to handle other types of method-handlers, such as MCP, Agents, and A2A.

In [1]:
from atomic_agentic.tools import Tool

## 1) `Tool` as an Adapter for Callables, Methods, and Functions
From a user perspective, the most foundational understanding is that a **`Tool` wraps a callable**, but instead of calling the function directly the way you would any other method, you execute the method through Tool using:

```python
tool.invoke({ ... }) # or tool.invoke(inputs = {...})
```

The Tool is responsible for:
- validating inputs
- binding them correctly to the underlying function
- executing the function
- returning the result

From the caller’s perspective, **every Tool is called the same way**.


In [2]:
def add(a: int, b: int = 0) -> int:
    """Add two integers and return the sum."""
    return a + b

add_tool = Tool(add)

result = add_tool.invoke({"a": 2, "b": 40})

f"Result: {result}"


'Result: 42'

At this point, a few important things should stand out:

- We never called `add(2, 40)` directly.
- We passed a **single dictionary** into `.invoke(...)`.
- The keys in the dictionary explicitly name the inputs.

This is the core Tool contract.


In [3]:
# Defaults still apply when omitted
add_tool.invoke({"a": 5})

5

## 2) The Single Dictionary Inputs Shape
While it may seem overkill to use a dictionary for calling methods wrapped in tools (in contrast to say enterin inputs in positionally), there are a few key benefits and solved problems that this design provides for Atomic Agentic's `Tool`.

First, it removes reliance on positional ordering, which is often fragile and hard to refactor.  
Making every call explicit is robust and and self-documenting.  
Most importantly, it ensures that *every* tool — regardless of it's method's header's shape — can be invoked in exactly the same way.

The underlying function signature can change, but the Tool interface remains stable.

In [4]:
def greet(name: str, *, excited: bool = False) -> str:
    """Return a greeting."""
    msg = f"Hello, {name}"
    return msg + "!!!" if excited else msg + "."

greet_tool = Tool(greet)

greet_tool.invoke({"name": "Ada"})


'Hello, Ada.'

In [5]:
# Ordering does not matter
greet_tool.invoke({"excited": True, "name": "Ada"})

'Hello, Ada!!!'

Regardless of whether a function uses positional parameters, keyword-only parameters, defaults, or a mix of all three, the **Tool is always invoked with a single mapping**.

This consistency is what allows Tools to scale across:
- local code
- orchestration layers
- LLM tool selection
- remote transports

Now that we understand how to *use* a Tool, we can start looking at how a Tool is structured internally.


## 3) The Anatomy of a Tool

Under the hood, every Tool can be understood as three cooperating parts:

1. **The callable itself**  
   This is the function or method that ultimately does the work.

2. **Input-handling logic**  
   This logic inspects the dictionary inputs and the callable’s signature and reorganizes the inputs into the correct positional arguments `args` and keyword arguments `kwargs` and passes them to the function to execute.

3. **Human- and LLM-readable metadata**  
   After initializing the Tool, it derives:
   - an argument map
   - a readable function signature
   - a name and description

These pieces together form the “bridge” between LLM reasoning and executable Python code.

### `arguments_map`: the Tool’s call plan

The **arguments_map** is the primary internal representation of how a Tool understands its inputs.

It is used to:
- determine which inputs are required
- apply default values
- bind mapping keys to positional or keyword arguments
- generate the human-readable signature
- support downstream systems (planners, workflows, agents)

If a parameter or return type is not annotated, the Tool defaults it to `"Any"`.


In [6]:
import json

def show_tool_plan(t: Tool) -> None:
    meta = t.to_dict()
    print("full_name:   ", t.full_name)
    print("name:        ", meta.get("name"))
    print("namespace:   ", meta.get("namespace"))
    print("description: ", meta.get("description"))
    print("signature:   ", meta.get("signature"))
    print("return type: ", meta.get("return_type"))
    print("arguments_map:")
    print(json.dumps(t.arguments_map, indent=2))

show_tool_plan(add_tool)


full_name:    Tool.default.add
name:         add
namespace:    default
description:  Add two integers and return the sum.
signature:    None
return type:  int
arguments_map:
{
  "a": {
    "index": 0,
    "kind": "POSITIONAL_OR_KEYWORD",
    "type": "int"
  },
  "b": {
    "index": 1,
    "kind": "POSITIONAL_OR_KEYWORD",
    "type": "int",
    "default": 0
  }
}


You'll notice that for each argument in our `arguments_map`, we provide the following details: index, kind, type, and default (if our inspection finds any). The `index` indicates the position in the function signature that the argument belongs, the `type` indicates the data type (int, dict, str, etc.), and the optional `default` indicates the value that it takes when inputs don't specify that argument.   
The `kind` categorizes the argument on whether or not the programmer needs to specify the name of argument that the value is associated with or if it needs to position the argument in the correct location. For instance, our `add_tool` could represent the same add method that does:

```python
'''
POSTITIONAL ONLY -> calls only accept with examples like:
add(1, 2)
add(3) # assumes a = 3, b = 0

Rejects any mention of the argument names:
add(a = 3, 2)
add(b = 1, a = 4)
...
'''
def add(a: int, b: int = 0, /) -> int:
    return a + b

'''
KEYWORD ONLY -> calls only accept with examples like:
add(a = 0, b = -1)
add(a = 2, b = 41)

Rejects any call that the doesn't use the keyword for the argument:
add(1, 3)
add(1, b = 2)
...
'''
def add(*, a: int, b: int = 0) -> int:
    return a + b


'''
MIXED SIGNATURES -> has positional only AND/OR keyword only or ambiguous.
Anything before '/' is a positional only argument, anything after '*' must
be a keyword only argument, and anything in between is either (provided 
positionals stay to the left and keywords to the right)

Accepts:
add_three(1, 2, c = 3)
add_three(1, b = 4, c = 6)
'''
def add_three(a:int, /, b:int, *, c:int = 2):
    return a + b + c

```



## 4) Bridging Tools & LLM's Through Natural Language Metadata

As mentioned in the beginning of this notebook, the `Tool` class serves as an adapter interface for humans & LLM's to percieve and interpret methods & functions for intelligent calling. While the `arguments_map` helps with inspecting inputs and constructing a signature, the context that an LLM or human can retrieve.

By default:
- the Tool’s **name** comes from the function name
- the Tool’s **description** comes from the function’s docstring (if present)

However, you can override either (or both) at Tool creation time.  
When you do so, you see the user provided metadata instead of the function’s own metadata.

In [7]:
def custom_operator(a: float, b: float) -> float:
    """Computes the ratio of a and b's sum to their product."""
    return (a + b) / (a * b)

inspected_tool = Tool(custom_operator)

print("======== Tool made from built-in inspection ========")
show_tool_plan(inspected_tool)

custom_tool = Tool(
    custom_operator,
    name="AWESOME_Custom_Operator",
    namespace="GnG",
    description="Calculates the quotient of a plus b and a times b."
)
print("======== Tool made with user-provided natural language parameters ========")
show_tool_plan(custom_tool)


======== Tool made from built-in inspection ========
full_name:    Tool.default.custom_operator
name:         custom_operator
namespace:    default
description:  Computes the ratio of a and b's sum to their product.
signature:    None
return type:  float
arguments_map:
{
  "a": {
    "index": 0,
    "kind": "POSITIONAL_OR_KEYWORD",
    "type": "float"
  },
  "b": {
    "index": 1,
    "kind": "POSITIONAL_OR_KEYWORD",
    "type": "float"
  }
}
======== Tool made with user-provided natural language parameters ========
full_name:    Tool.GnG.AWESOME_Custom_Operator
name:         AWESOME_Custom_Operator
namespace:    GnG
description:  Calculates the quotient of a plus b and a times b.
signature:    None
return type:  float
arguments_map:
{
  "a": {
    "index": 0,
    "kind": "POSITIONAL_OR_KEYWORD",
    "type": "float"
  },
  "b": {
    "index": 1,
    "kind": "POSITIONAL_OR_KEYWORD",
    "type": "float"
  }
}


The arguments maps are the same between these two methods, as the parameters are left unchanged, but the name, namespace, and description are all using what the user provided for our custom tool. This provides extra control to the user on what the LLMs need to see or understand about the tools we give them.  
You may have also noticed before, that there is also a `namespace` parameter, which effectively allows us as humans and the LLM to distinguish between methods with similar names but from different sources. It adds a layer of flexibility, leniency, and fine-grain distinctness between tools with minimal changes.  

We also can represent tools as strings directly, which composes all the natural language details into a single string, a massive boon for LLM-powered systems, as seen below: 

In [8]:
print("======== Tool Signatures ========")
print(inspected_tool.signature)
print(custom_tool.signature)
print()
print("======== Full Tool Strings ========")
print(inspected_tool)
print(custom_tool)

======== Tool Signatures ========
Tool.default.custom_operator(a: float, b: float) -> float
Tool.GnG.AWESOME_Custom_Operator(a: float, b: float) -> float

======== Full Tool Strings ========
<Tool.default.custom_operator(a: float, b: float) -> float - Computes the ratio of a and b's sum to their product.>
<Tool.GnG.AWESOME_Custom_Operator(a: float, b: float) -> float - Calculates the quotient of a plus b and a times b.>


## 5) Looking ahead

You’ve now seen that:

- Tools standardize invocation through a dict-first interface
- Input inspection and metadata generation are tightly coupled
- The same Tool abstraction works for local and remote execution

In future notebooks, we’ll build on this foundation to explore:
- Acting as proxies to MCP server tools
- Agents and AgentTools
- Tool-using agents and planners
- A2A-based agent communication
